**THIS FILE IS OUT OF DATE**. It will be updated shortly.

In [1]:
import numpy as np
import pandas as pd

In [2]:
from res_ind_lib import *

# Load the data

In [3]:
#Load the excel data file into a dataframe (table)
df=pd.read_excel("inputs/all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the first column (Provinces) as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variables and the 3rd (2) row is empty. We skip them
                ) 
df.index.name="province" #rename the index to "province" for clarity

df = def_ref_values(df).dropna()

#Displays the first rows of that dataframe
df.head()

,gdp_pc_pp,pop,pov_head,shew,gdp_pc_pp_nat,rel_gdp_pp,share1,national_pop,axfin_p,axfin_r,...,T_rebuild_K,pi,nat_buyout,sigma_r,sigma_p,income_elast,rho,protectionref,gdp_pc_pp_ref,v_s
province,,,,,,,,,,,,,,,,,,,,,
Abra,31.622466,240135.244121,0.373595,0.953416,42.034627,0.752296,0.509132,83650215.459496,0.693233,0.693233,...,3,0.2,0.3,0,0,1.5,0.05,10.57,31.622466,0.100000
Agusan Del Norte,43.726667,661728.454375,0.346715,0.821277,42.034627,1.040253,0.368196,83650215.459496,0.496880,0.496880,...,3,0.2,0.3,0,0,1.5,0.05,9.41,43.726667,0.100000
Agusan Del Sur,30.897436,677779.682154,0.480785,0.821277,42.034627,0.735047,0.521079,83650215.459496,0.475969,0.475969,...,3,0.2,0.3,0,0,1.5,0.05,8.61,30.897436,0.100000
Aklan,30.515652,554414.442422,0.249662,0.823003,42.034627,0.725965,0.527598,83650215.459496,0.660083,0.660083,...,3,0.2,0.3,0,0,1.5,0.05,2.00,30.515652,0.135089
Albay,33.793313,1264097.894966,0.409587,0.800000,42.034627,0.803940,0.476426,83650215.459496,0.551314,0.551314,...,3,0.2,0.3,0,0,1.5,0.05,2.00,33.793313,0.100000


 # Define all parameters and variables

## Description of inequalities

In [4]:
#proportion of poor people (poverty incidence)
ph = df["pov_head"]

#income in the province
gdp_pc_pp = df["rel_gdp_pp"] * df["gdp_pc_pp_nat"] 
gdp_pc_pp.head()

#consumption levels for poor and non-poor
cp=   df["share1"] *gdp_pc_pp
cr=  (1 - df["pov_head"]*df["share1"])/(1-df["pov_head"]) *gdp_pc_pp

## Exposure

In [5]:
#exposure for poor and nonpoor
fap =df["fap"]
far =df["far"]

## Vulnerability

In [6]:
#early-warning-adjusted vulnerability: access to early warning and reactivity to early warnings reduce the effective vulnerability
vp = df["v_p"]*(1-df["pi"]*df["shew"])
vr = df["v_r"]*(1-df["pi"]*df["shew"])

In [7]:
#losses shared within the province: this value is used to scale down transfers
v_shared = df["v_s"] *(1-df["pi"]*df["shew"]) 

## Ex-post support

![](img/scale_up.png)

In [8]:
#fraction of losses shared taking into account diversification lambda and scaleup of diversified income sigma
tot_p=1-(1-df["social_p"])*(1-df["sigma_p"])
tot_r=1-(1-df["social_r"])*(1-df["sigma_r"])


## Link between asset losses and discounted consumption losses

We define $\Gamma$, the ratio of discounted consumption losses to asset to losses.
The calculus of $\Gamma$ depends on the given a discount rate $\rho$ and the time $N$ it taks to rebuild 95% of asset losses, and takes into account that some consumption  has to be diverted for reconstruction.

![](img/reconstruction.png)

Note the two extreme cases:
- As $N$ tends toward zero, $\Gamma$ tends to 1: consumption losses equal asset losses.
- As $N$ tends toward the infinity (no reconstruction), $\Gamma$ tends to $\mu/\rho$.


In [9]:
#time it takes to rebuild the damaged asset
N= df["T_rebuild_K"]

#discount rate
rho = df["rho"]

#productivity of capital
mu= df["avg_prod_k"]

# Link between immediate and discounted losses
gamma =(mu +3/N)/(rho+3/N) 
gamma.head()

province
Abra                1.171429
Agusan Del Norte    1.171429
Agusan Del Sur      1.171429
Aklan               1.171429
Albay               1.171429
dtype: float64

## From consumption losses to welfare losses

This parameter determines the concavity of the utility function

In [10]:
#Eta
elast=  df["income_elast"]
elast.head()

province
Abra                1.5
Agusan Del Norte    1.5
Agusan Del Sur      1.5
Aklan               1.5
Albay               1.5
Name: income_elast, dtype: float64

# Compute asset losses dK and welfare losses dW
The following function computes the average asset losses per person in the event of a disaster (dK), and the average welfare losses in the event of a disaster (dW). 

In [11]:
#Welfare losses 
delta_W,dK,dcap,dcar =calc_delta_welfare(ph,fap,far,vp,vr,v_shared,cp,cr,tot_p,tot_r,mu,gamma,rho,elast)



In the following we look at how this function works, step by step.

# calc_delta_welfare

In [12]:
# fractions of people non-poor/poor affected/non affected over total pop

#poor
nap= ph*fap #directly affected
nnp=ph*(1-fap ) #not directly affected

#nonpoor
nar=(1-ph)*far
nnr=(1-ph)*(1-far)

In [13]:
#capital estimate from consumption and productivity
kp = cp/mu
kr = cr/mu

In [14]:
#total capital losses
dK = kp*vp*fap*ph+\
     kr*vr*far*(1-ph)

In [15]:
# current consumption losses per category of population
d_cur_cnp=fap*v_shared *tot_p *kp    #among population not directly affected by the disaster
d_cur_cnr=far*v_shared *tot_r *kr
d_cur_cap=vp*(1-tot_p)*kp + d_cur_cnp # among population directly affected by the disaster 
d_cur_car=vr*(1-tot_r)*kr + d_cur_cnr

In [16]:
#losses in NPV after reconstruction 
d_npv_cnp= gamma*d_cur_cnp    #among population not directly affected by the disaster
d_npv_cnr= gamma*d_cur_cnr
d_npv_cap= gamma*d_cur_cap    #among population directly affected by the disaster 
d_npv_car= gamma*d_cur_car

In [17]:
#welfare before the disaster. We assume wefare depends on discounted  consumption flux
Wpre =ph* welf(cp/rho,elast) + (1-ph)*welf(cr/rho,elast)

# welfare after the disaster
Wpost=  nap*welf(cp/rho-d_npv_cap,elast) + \
        nnp*welf(cp/rho-d_npv_cnp,elast) + \
        nar*welf(cr/rho-d_npv_car,elast) + \
        nnr*welf(cr/rho-d_npv_cnr,elast)

#welfare losses
delta_W =Wpre -Wpost 



In [19]:
df["delta_W"]=delta_W
df["dK"]=dK

# calc_risk_and_resilience_from_k_w

In [21]:
calc_risk_and_resilience_from_k_w(df)[["risk","resilience"]].head()

,risk,resilience
province,,
Abra,0.012783,0.457644
Agusan Del Norte,0.031242,0.525445
Agusan Del Sur,0.018650,0.384153
Aklan,0.000000,NaN
Albay,0.000000,NaN


In [29]:
#To express welfare losses in currenncy, we use the W'(c)
h=1e-4
wprime =(welf(df["gdp_pc_pp_nat"]/rho+h,df["income_elast"])-welf(df["gdp_pc_pp_nat"]/rho-h,df["income_elast"]))/(2*h)

#reference losses
dWref   = wprime*df["dK"]

In [30]:
#expected welfare losses per capita, epxressed in currency
df["dWpc_curency"] =  (dW)/wprime /df["protection"]
df["dWpc_curency"].head()

province
Abra                0.404221
Agusan Del Norte    1.366094
Agusan Del Sur      0.576242
Aklan               0.000000
Albay               0.000000
Name: dWpc_curency, dtype: float64

In [31]:
#risk to wellbeing as a fraction of GDP
df["risk"]= df["dWpc_curency"]/(df["gdp_pc_pp_ref"])  
df["risk"].head()

province
Abra                0.012783
Agusan Del Norte    0.031242
Agusan Del Sur      0.018650
Aklan               0.000000
Albay               0.000000
Name: risk, dtype: float64

In [32]:
#total expected welfare losses due to disaster
df["dWtot_currency"]=df["dWpc_curency"]*df["pop"] 
df["dWtot_currency"].head()

province
Abra                 97067.669262
Agusan Del Norte    903982.977995
Agusan Del Sur      390565.404291
Aklan                    0.000000
Albay                    0.000000
Name: dWtot_currency, dtype: float64

In [35]:
#SOCIO-ECONOMIC CAPACITY (Resilience)
df["resilience"]=dWref/(df["delta_W"] );
df["resilience"].head()

province
Abra                0.457644
Agusan Del Norte    0.525445
Agusan Del Sur      0.384153
Aklan                    NaN
Albay                    NaN
Name: resilience, dtype: float64

In [36]:
#RISK TO ASSETS
df["risk_to_assets"]  =df.resilience* df.risk
df["risk_to_assets"].head()

province
Abra                0.005850
Agusan Del Norte    0.016416
Agusan Del Sur      0.007165
Aklan                    NaN
Albay                    NaN
Name: risk_to_assets, dtype: float64